# 🚀 머신러닝 실습 : 고객 구매 데이터로 성별 예측 모델링 (분류 문제)

* 주어진 데이터는 백화점 고객의 1년 간 구매 데이터입니다.
* 고객 3,500명에 대한 학습용 데이터(y.csv, X.csv)를 이용하여 성별예측 모형을 만들어보세요.
* 모델의 성능은 자유롭게 측정해봅니다!

## [실습 프로세스]
1. 데이터 불러오기  
2. 데이터 탐색
3. 데이터 전처리  
4. 학습/테스트 데이터 분리  
5. 모델 선택 및 학습  
6. 예측 및 평가  


<br/>

---

<br/>
<br/>

# 0. 라이브러리 불러오기

* 라이브러리를 가져와서 과정을 준비합니다

In [99]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

<br/>

---

<br/>
<br/>

# 1. 데이터 불러오기
* 데이터를 가져와서 과정을 준비합시다.
- 인코딩 방식은 'euc-kr' 을 활용하세요.
- 데이터 출처 : 한국데이터산업진흥원 빅데이터분석기사 실기 공개 예시 문항

- 독립 변수 데이터셋 : ./data/X.csv
- 종속 변수 데이터셋 : ./data/y.csv


데이터 파일을 불러옵니다. 보통 CSV 파일을 pandas로 읽어옵니다.

In [100]:
import os
# 노트북 파일이 있는 폴더로 이동 (예시)
os.chdir(r'C:\githome\hipython_rep')

# 변경 후 확인
print("변경 후:", os.getcwd())

변경 후: C:\githome\hipython_rep


In [101]:
X = pd.read_csv('./data1/X.csv', encoding='euc-kr')
y = pd.read_csv('./data1/y.csv', encoding='euc-kr')

<br/>

---

<br/>
<br/>

# 2. 데이터 탐색하기
* 데이터를 이해할 수 있도록 탐색과정을 수행해봅시다.


데이터의 상위 몇 개 행을 출력하여 전체 구조를 미리 확인합니다.


데이터의 요약 정보나 통계 정보를 출력해 변수들의 유형과 분포를 확인합니다.


데이터의 요약 정보나 통계 정보를 출력해 변수들의 유형과 분포를 확인합니다.

In [102]:
X.head(3)

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1


In [103]:
y.head(3)

,cust_id,gender
0,0,0
1,1,0
2,2,1


In [104]:
X.isna().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

In [105]:
temp = X.groupby('주구매상품')['환불금액'].mean()
temp

주구매상품
가공식품      1.979173e+07
가구        1.011060e+07
건강식품      1.277410e+07
골프        2.146390e+07
구두        3.320762e+06
기타        1.719766e+07
남성 캐주얼    6.450662e+06
남성 트랜디    2.512380e+07
남성정장      4.507000e+06
농산물       2.263326e+07
대형가전      9.732867e+07
디자이너      7.049435e+07
란제리/내의    2.240960e+07
명품        3.892411e+07
모피/피혁     5.531688e+06
보석                 NaN
생활잡화      6.528000e+06
섬유잡화      6.353624e+06
셔츠        3.964160e+06
소형가전               NaN
수산품       8.470131e+06
스포츠       9.834013e+06
시티웨어      1.811968e+07
식기        4.255000e+07
아동        3.021707e+07
악기                 NaN
액세서리      6.888000e+06
육류        1.163039e+07
일용잡화      9.641162e+06
젓갈/반찬     1.470748e+07
주류        5.600000e+03
주방가전      9.446430e+07
주방용품      6.425075e+06
차/커피      6.920356e+06
축산가공      1.087173e+07
침구/수예     3.175245e+07
캐주얼       2.077980e+07
커리어       2.447445e+07
통신/컴퓨터             NaN
트래디셔널     6.542200e+06
피혁잡화      3.356925e+06
화장품       2.198165e+07
Name: 환불금액, dtype: float64

In [106]:
X['주구매상품'].head()

0        기타
1       스포츠
2    남성 캐주얼
3        기타
4        보석
Name: 주구매상품, dtype: object

<br/>

---

<br/>
<br/>

# 3. 데이터 전처리
* 전처리 과정을 통해서 머신러닝에 사용할 수 있는 형태의 데이터 준비

* 단순히 1부터의 숫자를 부여한 'cust_id'를 수치형 변수로 받아들이면, 결과가 왜곡될 수 있으니 컬럼을 제거합니다.

- 데이터에 결측치가 있는지 확인해보세요


- 결측치에 0으로 채워 넣어 모델 학습에 지장이 없도록 합니다.


문자형 범주 데이터를 숫자로 바꾸기 위한 인코딩을 수행합니다.

각 데이터에 표준화를 적용하여 데이터의 스케일(크기 차이)을 맞춰줍니다.
- 평균을 0, 표준편차를 1로 맞춰서 → 데이터가 정규 분포 형태로 변환되도록 하세요

In [107]:
temp = X.groupby('주구매상품')['환불금액'].transform('mean')
X['환불금액'] = X['환불금액'].mask(X['환불금액'].isna(), temp)

In [108]:
X['환불금액'] = X['환불금액'].fillna(X['환불금액'].mean())
X.isna().sum().sum()

np.int64(0)

In [109]:
X.corr(numeric_only=True)

,cust_id,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기
cust_id,1.000000,-0.008004,-0.005883,0.036162,-0.025234,-0.004840,0.001900,0.020070
총구매액,-0.008004,1.000000,0.700080,0.366023,0.659084,0.090022,0.014396,-0.212944
최대구매액,-0.005883,0.700080,1.000000,0.356716,0.374147,0.018980,0.022277,-0.115837
환불금액,0.036162,0.366023,0.356716,1.000000,0.237056,-0.040541,-0.038327,-0.086858
내점일수,-0.025234,0.659084,0.374147,0.237056,1.000000,0.225264,-0.010325,-0.293200
내점당구매건수,-0.004840,0.090022,0.018980,-0.040541,0.225264,1.000000,0.007659,-0.091151
주말방문비율,0.001900,0.014396,0.022277,-0.038327,-0.010325,0.007659,1.000000,0.003372
구매주기,0.020070,-0.212944,-0.115837,-0.086858,-0.293200,-0.091151,0.003372,1.000000


In [110]:
temp = y['gender'].value_counts(normalize=True)
temp

gender
0    0.624
1    0.376
Name: proportion, dtype: float64

In [111]:
A = X['주구매지점'].unique()
A

array(['강남점', '잠실점', '관악점', '광주점', '본  점', '일산점', '대전점', '부산본점', '분당점',
       '영등포점', '미아점', '청량리점', '안양점', '부평점', '동래점', '포항점', '노원점', '창원점',
       '센텀시티점', '인천점', '대구점', '전주점', '울산점', '상인점'], dtype=object)

In [112]:
B = X['주구매상품'].unique()
B

array(['기타', '스포츠', '남성 캐주얼', '보석', '디자이너', '시티웨어', '명품', '농산물', '화장품',
       '골프', '구두', '가공식품', '수산품', '아동', '차/커피', '캐주얼', '섬유잡화', '육류',
       '축산가공', '젓갈/반찬', '액세서리', '피혁잡화', '일용잡화', '주방가전', '주방용품', '건강식품',
       '가구', '주류', '모피/피혁', '남성 트랜디', '셔츠', '남성정장', '생활잡화', '트래디셔널',
       '란제리/내의', '커리어', '침구/수예', '대형가전', '통신/컴퓨터', '식기', '소형가전', '악기'],
      dtype=object)

In [113]:
X['주구매지점'] = X['주구매지점'].astype('category').cat.codes
X['주구매상품'] = X['주구매상품'].astype('category').cat.codes

<br/>

---

<br/>
<br/>

# 5-1. 모델링 - LogisticRegression

* 본격적으로 모델을 선언하고 학습시킵니다.


필요한 라이브러리를 불러옵니다.

모델을 선언하여 객체화시킵니다.


모델을 학습 데이터에 맞춰 학습시킵니다.

In [114]:
from sklearn.linear_model import LogisticRegression

In [115]:
lr_clf = LogisticRegression(solver='liblinear')
knn_clf = KNeighborsClassifier(n_neighbors=7)
vo_clf = VotingClassifier(estimators=[('LR', lr_clf),('KNN', knn_clf)],
                          voting='soft')

In [116]:
vo_clf.fit(X_train, y_train)
vo_pred = vo_clf.predict(X_test)
accuracy_score(y_test, vo_pred)

0.6214285714285714

In [117]:
confusion_matrix(y_test, vo_pred)

array([[363,  88],
       [177,  72]])

In [118]:
print(classification_report(y_test, vo_pred))

              precision    recall  f1-score   support

           0       0.67      0.80      0.73       451
           1       0.45      0.29      0.35       249

    accuracy                           0.62       700
   macro avg       0.56      0.55      0.54       700
weighted avg       0.59      0.62      0.60       700



# 6-1. 예측 성능 확인해보기 - LogisticRegression

- 학습된 모델로 테스트 데이터에 대한 예측을 수행합니다.

- 학습시킨 모델의 성능을 알아봅니다
- 각 평가지표로 모델의 성능을 수치화하여 확인합니다.
- 필요한 라이브러리를 import 하고 성능을 확인해보세요 (정확도, 정밀도, 재현율, f1, confusion_matrix)

In [119]:
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)

    print(confusion)
    print(accuracy, precision, recall)

In [120]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

models = {
  'KNN': KNeighborsClassifier(n_neighbors=5),
  'RF': RandomForestClassifier(500, max_depth=6,random_state=0),
  'LR': LogisticRegression(max_iter=3000)
}
for name, model in models.items():
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  acc = accuracy_score(y_test, y_pred)
  print(f'{name} 모델의 정확도: {acc:.5f}')

KNN 모델의 정확도: 0.63143
RF 모델의 정확도: 0.65714
LR 모델의 정확도: 0.64429


In [121]:
pred_proba = vo_clf.predict_proba(X_test) #prova에 X_test를 넣었을 때 0일 확률, 1일 확률
pos_proba = pred_proba[:,1] # 양성클래스일 확률
threshold = 0.4 # 임계치
custom_proba = (pos_proba >= threshold).astype(int) # 임계치보다 크면 1로 넣는다.
confusion_matrix(y_test, custom_proba)

# 정확도, 정밀도, 재현율
get_clf_eval(y_test, custom_proba)

[[265 186]
 [108 141]]
0.58 0.43119266055045874 0.5662650602409639



<br/>

---

<br/>
<br/>

# 5-2. 모델링 - DecisionTreeClassifier

* 본격적으로 모델을 선언하고 학습시킵니다.


필요한 라이브러리를 불러옵니다.

모델을 선언하여 객체화시킵니다.

모델을 학습 데이터에 맞춰 학습시킵니다.

In [122]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth': [6],
    'min_samples_split' : [2,5],
    'min_samples_leaf': [1,6]
}

rf_clf = RandomForestClassifier(
    n_estimators=100, 
    random_state=0, 
    n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, 
                       param_grid=params, 
                       cv=2, 
                       n_jobs=-1)
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=2, estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
             n_jobs=-1,
             param_grid={'max_depth': [6], 'min_samples_leaf': [1, 6],
                         'min_samples_split': [2, 5]})

In [123]:
grid_cv.best_params_, grid_cv.best_score_

({'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 5},
 np.float64(0.65))

In [124]:
from sklearn.metrics import accuracy_score
best_model = grid_cv.best_estimator_
best_pred = best_model.predict(X_test)
accuracy_score(y_test, best_pred)

0.6457142857142857



<br/>
<br/>

# 6-2. 예측 성능 확인해보기 - DecisionTreeClassifier

- 학습된 모델로 테스트 데이터에 대한 예측을 수행합니다.

- 학습시킨 모델의 성능을 알아봅니다
- 각 평가지표로 모델의 성능을 수치화하여 확인합니다.
- 필요한 라이브러리를 import 하고 성능을 확인해보세요 (정확도, 정밀도, 재현율, f1, confusion_matrix)

In [154]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [155]:
# 1. 모델 생성 (최대 깊이 제한 걸어서 과적합 방지)
dt_clf = DecisionTreeClassifier(max_depth=5, random_state=42)

# 2. 학습
dt_clf.fit(X_train, y_train)

# 3. 예측
dt_pred = dt_clf.predict(X_test)

# 4. 정확도 평가
acc = accuracy_score(y_test, dt_pred)
print(f'Decision Tree 정확도: {acc * 100:.2f}%')

# 5. 정밀도, 재현율 등 추가 지표 출력
print("\n[분류 리포트]")
print(classification_report(y_test, dt_pred))

# 6. 혼동 행렬 확인
print("\n[혼동 행렬]")
print(confusion_matrix(y_test, dt_pred))


Decision Tree 정확도: 64.57%

[분류 리포트]
              precision    recall  f1-score   support

           0       0.69      0.81      0.75       451
           1       0.50      0.35      0.41       249

    accuracy                           0.65       700
   macro avg       0.60      0.58      0.58       700
weighted avg       0.63      0.65      0.63       700


[혼동 행렬]
[[365  86]
 [162  87]]




<br/>
<br/>

# 6-3. 예측 성능 확인해보기 - RandomForestClassifier

- 학습된 모델로 테스트 데이터에 대한 예측을 수행합니다.

- 학습시킨 모델의 성능을 알아봅니다
- 각 평가지표로 모델의 성능을 수치화하여 확인합니다.
- 필요한 라이브러리를 import 하고 성능을 확인해보세요 (정확도, 정밀도, 재현율, f1, confusion_matrix)

In [153]:
from sklearn.model_selection import GridSearchCV

# 1. GridSearch로 최적 파라미터 찾기
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=5)
grid_cv.fit(X_train, y_train)

# 2. 최적 파라미터 확인
print(grid_cv.best_params_)  # 여기에 기반해 아래 모델 설정

# 3. 최적 파라미터로 모델 재학습 (지금 질문한 코드)
rf = RandomForestClassifier(random_state=42, 
                            n_estimators=350, 
                            max_depth=20, 
                            min_samples_split=3, 
                            n_jobs=-1)

rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
accuracy_score(y_test, rf_pred)

# 4. 최종 정확도 평가
print(f'최종 모델 정확도: {accuracy_score(y_test, rf_pred)}')


{'max_depth': 6, 'min_samples_leaf': 6, 'min_samples_split': 2}
최종 모델 정확도: 0.6228571428571429




<br/>
<br/>

# 6-4. 예측 성능 확인해보기 - XGBoost

- 학습된 모델로 테스트 데이터에 대한 예측을 수행합니다.

- 학습시킨 모델의 성능을 알아봅니다
- 각 평가지표로 모델의 성능을 수치화하여 확인합니다.
- 필요한 라이브러리를 import 하고 성능을 확인해보세요 (정확도, 정밀도, 재현율, f1, confusion_matrix)

In [132]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb = XGBClassifier(n_estimators=300, max_depth=6, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)
print(f'정확도: {accuracy_score(y_test, pred) * 100:.2f}%')


정확도: 59.00%


In [133]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.67      0.73      0.70       451
           1       0.41      0.33      0.37       249

    accuracy                           0.59       700
   macro avg       0.54      0.53      0.53       700
weighted avg       0.57      0.59      0.58       700



In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_cols = ['주구매지점', '주구매상품'] 
ct = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
], remainder='passthrough')

X_train_encoded = ct.fit_transform(X_train)
X_test_encoded = ct.transform(X_test)


In [136]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)
print(f'정확도: {accuracy_score(y_test, pred) * 100:.2f}%')


정확도: 59.00%


<br/>

---


<br/>

## 7.  위 4가지 모델의 학습 & 예측 & 평가 결과를 확인하고 최고 성능을 내는 모델을 찾아봅시다!

- 어떤 모델이 가장 성능이 좋은가요 ?
DecisionTreeClassifier : 64%
LogisticRegression : 62%
RandomForestClassifier : 62%
XGBClassifier : 59%